New Comments Extraction
===

This script processes the json comments in the new (2019) dataset to a CSV file containing the barebones interaction info.


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import re
import pandas as pd
import numpy as np

from collections import Counter, defaultdict
import sqlite3
from nltk import word_tokenize
from html.parser import HTMLParser
from tqdm import tqdm
import random
import pickle
import json

from datetime import datetime
from pprint import pprint

import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib
import pylab as pl
from IPython.core.display import display, HTML

In [3]:
from pathlib import Path
git_root_dir = !git rev-parse --show-toplevel
git_root_dir = Path(git_root_dir[0].strip())
git_root_dir

PosixPath('/panfs/roc/groups/3/srivbane/levon003/repos/sna-social-support')

In [4]:
import sys
caringbridge_core_path = "/home/srivbane/levon003/repos/caringbridge_core"
sys.path.append(caringbridge_core_path)

In [5]:
import cbcore.data.paths as paths
import cbcore.data.dates as dates

In [6]:
raw_data_dir = paths.raw_data_2019_filepath

In [7]:
working_dir = "/home/srivbane/shared/caringbridge/data/projects/sna-social-support/csv_data"
assert os.path.exists(working_dir)

In [8]:
output_filepath = os.path.join(working_dir, "new_comments_metadata_raw.csv")

In [9]:
def extract_long(json_value):
    if type(json_value) == int:
        return json_value
    elif type(json_value) == str:
        return int(json_value)
    elif type(json_value) == dict:
        return int(json_value['$numberLong'])
    elif json_value == 0.0:  # special case for userId, which I assume is essentially "missing/unknown" or user deleted?
        return ""
    elif type(json_value) == float:
        return int(json_value)
    else:
        raise ValueError(f"Type '{type(json_value)}' not implemented: {str(json_value)}")

In [10]:
comment_parent_counts = defaultdict(int)
comments_filepath = os.path.join(raw_data_dir, 'comment_scrubbed.json')
with open(output_filepath, 'w') as outfile:
    with open(comments_filepath, encoding='utf-8') as infile:
        for line in tqdm(infile, total=31052715):
            comment = json.loads(line)
            comment_parent_counts[comment['parentType']] += 1
            journal_oid = comment['ancestorId']
            site_id = extract_long(comment['siteId'])
            user_id = extract_long(comment['userId'])
            created_at = dates.get_date_from_json_value(comment['createdAt'])
            updated_at = dates.get_date_from_json_value(comment['updatedAt'])
            
            outfile.write(f"{user_id},{site_id},comment,{journal_oid},{created_at},{updated_at}\n")
comment_parent_counts

100%|██████████| 31052715/31052715 [46:17<00:00, 11181.58it/s]


defaultdict(int, {'journal': 30853635, 'comment': 199080})